In [ ]:
import pickle
import random
import numpy as np
import sys
from pathlib import Path
import importlib

# Or safer for notebooks where __file__ is not defined:
sys.path.append(str(Path().resolve().parent / "src"))

import golf_utils
import golf_scoring
importlib.reload(golf_scoring)

import golf_utils

from golf_utils import postprocess_players
import golf_scoring
from collections import defaultdict
from golf_classes import Player, PlayerRoundInfo, Tournament, Round, Team
from golf_scoring import compute_real_stableford, simulate_round,  stableford_points, calibrated_simulate_round, hole_pars


In [ ]:
def simulate_tournament(cteams, scoring_func, num_simulations=10000):
    for cteam in cteams:
        cteam.simulated_scores = []

    # Run simulations
    for _ in range(num_simulations):
        sim_results = []
        for cteam in cteams:
            sim_score = cteam.simulate_score(scoring_func)
            cteam.simulated_scores.append(sim_score)
            sim_results.append((cteam, sim_score))

        # Sort to find placements
        sim_results.sort(key=lambda x: x[1], reverse=True)
        winner = sim_results[0][0]
        top3 = [res[0] for res in sim_results[:3]]

        # Track placement counts
        winner.placement_probs['win'] = winner.placement_probs.get('win', 0) + 1
        for team in top3:
            team.placement_probs['top3'] = team.placement_probs.get('top3', 0) + 1

    # Normalize counts to probabilities
    for cteam in cteams:
        cteam.placement_probs['win'] = cteam.placement_probs.get('win', 0) / num_simulations
        cteam.placement_probs['top3'] = cteam.placement_probs.get('top3', 0) / num_simulations
